### El objetivo de este Jupyter es encontrar las cordenadas  de los pisos más cercanos a la dirección que se incluya. Estas funciones son finalmente incluidas en el fichero "get_address_neigh.py".

In [3]:
from dotenv import load_dotenv
import requests
load_dotenv()
import os
import geopandas as gpd
import pandas as pd
import fiona

In [4]:
def get_coordinates(address):
    key = os.getenv('GOOGLEAUTH')
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
    rqst  = requests.get(url + address + '&key=' + key)    
    datos = rqst.json()
    location = datos['results'][0]['geometry']['location']
    print(url + address + '&key=' + key)
    return location


In [ ]:
x = get_coordinates("calle de gargantilla madrid")
x

In [25]:
x["lat"]

40.3988018

In [6]:
df = gpd.GeoDataFrame({'lat': [x["lat"]], 'lng': [x["lng"]]})

In [7]:
#Crear GeoDataFrame a partir de un DataFrame con coordenadas
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng, df.lat),crs="EPSG:4326")
gdf = gdf.to_crs("EPSG:4326")

In [8]:
print('Nuevo CRS: ',gdf.crs)

Nuevo CRS:  EPSG:4326


In [9]:
barrios = gpd.read_file("BARRIOS.shp")
barrios = barrios.to_crs("EPSG:4326")
print('Nuevo CRS: ',barrios.crs)

Nuevo CRS:  EPSG:4326


In [10]:
barrios_gdf.head(1)


NameError: name 'barrios_gdf' is not defined

In [11]:
barrio_loc = gpd.sjoin(gdf, barrios, how="left", op='within')

In [12]:
barrio_loc

,lat,lng,geometry,index_right,OBJECTID,geodb_oid,CODDIS,NOMDIS,CODBAR,CODDISTRIT,CODBARRIO,NOMBRE,ORIG_FID
0,40.398802,-3.70408,POINT (-3.70408 40.39880),39,8,8,02,Arganzuela,022,02,2-2,Acacias,7


In [13]:
distrito = barrio_loc["NOMBRE"].to_string(index=False).lstrip()
distrito

'Acacias'

In [21]:
x = pd.read_csv("barrios_puntuaciones.csv")
#x =x.drop_duplicates(subset=["NOMBRE"])

In [22]:
x = x[x["NOMBRE"]==distrito]
x

,Unnamed: 0,NOMDIS,CODBAR,CODDISTRIT,NOMBRE,security,transport,health,education
112,10507,Arganzuela,022,02,Acacias,1,137,23,3


In [19]:
int(x["health"].to_string(index=False).lstrip())

23